In [31]:
from sklearn import datasets
import numpy as np
import pandas as pd
import math

### داده ها را load میکنیم و سپس correlation آن ها را میشکیم که نشان میدهد هیچ دو ویژگی با correlation خیلی بالا وجود ندارد پس داده ها را به همانگونه که هستند نگه میداریم.

In [3]:
wine_data = datasets.load_wine()

In [9]:
pd_wine = pd.DataFrame(wine_data.data)
pd_wine['target'] = wine_data.target
possible_classes = list(dict.fromkeys(wine_data.target))

corr = pd_wine.corr()
corr.style.background_gradient(cmap='coolwarm')

,0,1,2,3,4,5,6,7,8,9,10,11,12,target
0,1.000000,0.094397,0.211545,-0.310235,0.270798,0.289101,0.236815,-0.155929,0.136698,0.546364,-0.071747,0.072343,0.643720,-0.328222
1,0.094397,1.000000,0.164045,0.288500,-0.054575,-0.335167,-0.411007,0.292977,-0.220746,0.248985,-0.561296,-0.368710,-0.192011,0.437776
2,0.211545,0.164045,1.000000,0.443367,0.286587,0.128980,0.115077,0.186230,0.009652,0.258887,-0.074667,0.003911,0.223626,-0.049643
3,-0.310235,0.288500,0.443367,1.000000,-0.083333,-0.321113,-0.351370,0.361922,-0.197327,0.018732,-0.273955,-0.276769,-0.440597,0.517859
4,0.270798,-0.054575,0.286587,-0.083333,1.000000,0.214401,0.195784,-0.256294,0.236441,0.199950,0.055398,0.066004,0.393351,-0.209179
5,0.289101,-0.335167,0.128980,-0.321113,0.214401,1.000000,0.864564,-0.449935,0.612413,-0.055136,0.433681,0.699949,0.498115,-0.719163
6,0.236815,-0.411007,0.115077,-0.351370,0.195784,0.864564,1.000000,-0.537900,0.652692,-0.172379,0.543479,0.787194,0.494193,-0.847498
7,-0.155929,0.292977,0.186230,0.361922,-0.256294,-0.449935,-0.537900,1.000000,-0.365845,0.139057,-0.262640,-0.503270,-0.311385,0.489109
8,0.136698,-0.220746,0.009652,-0.197327,0.236441,0.612413,0.652692,-0.365845,1.000000,-0.025250,0.295544,0.519067,0.330417,-0.499130
9,0.546364,0.248985,0.258887,0.018732,0.199950,-0.055136,-0.172379,0.139057,-0.025250,1.000000,-0.521813,-0.428815,0.316100,0.265668


In [23]:
class_counts = pd_wine.groupby(['target']).count()
prior = {}
for i in range(len(possible_classes)):
    prior[i] = class_counts.iloc[i, 0]/len(pd_wine)
    

### تابع زیر ضرب احتمال هرکدام از ویژگی های داده تست به شرط اینکه عضو کلاس داده شده باشند را محاسبه میکند

In [115]:
def calculate_probs(train_data, test_data, cls):
    for i in range(len(wine_data.data[0])-1):
        selected = train_data.loc[train_data['target'] == cls].iloc[:, i].to_numpy()
        mean, var = np.mean(selected), np.var(selected)
        result = 1 / math.sqrt(2*math.pi*(var **2))
        result *= math.exp(-((test_data[i]-mean)**2)/(2*var**2))
    return result
        
        
    

### در این کلاس یک سری داده آموزشی و یک داده تست گرفته میشود و با استفاده از تابع قبلی کلاس آن تخمین زده میشود. کلاسی که بیشترین احتمال بین همه کلاس ها را دارد به عنوان خروجی برگردانده میشود. 

In [116]:
def predict(train_data, test_data):
    probs = prior.copy()
    maxi = 0
    pred_class = 0
    for cls in possible_classes:
        probs[cls]*=calculate_probs(train_data, test_data, cls)
        if probs[cls] > maxi:
            maxi = probs[cls]
            pred_class = cls
    return pred_class
    

### داده ها به k بخش تقسیم میشوند برای k-fold-cross-validation

In [117]:
def k_fold_cross_validation(data, k):
    shuffle_data = pd_wine.sample(frac=1)
    m = len(shuffle_data)
    folds = []
    for i in range(k):
        folds.append(shuffle_data.iloc[int(m*i/k): int(m*(i+1)/k), :])
    return folds
    

### برای 6 بخش بدست آمده عملیات انجام میشود و در مجموع تعداد درست ها تقسیم بر کل داده ها میشود. معادل میانگیری دقت برای هر fold.

In [118]:
X = pd_wine.iloc[:, :-1]
Y = pd_wine.iloc[:, -1]
folds = k_fold_cross_validation(pd_wine, 6)
correct = 0.0
temp_acc = 0
for i in range(6):
    train_temp = pd.concat([folds[1], folds[2], folds[3], folds[4], folds[5]])
    test_temp = folds[0].copy()
    folds[0] = folds[(i+1)%6].copy()
    folds[(i+1)%6] = test_temp.copy()
    for index, row in test_temp.iterrows():
        mod_row = row.to_numpy()
        predicted = predict(train_temp, mod_row)
        if mod_row[-1] not in (1,2,0):
            print('errorrrr')
        if predicted == mod_row[-1]:
            correct += 1
    temp_acc = 
acc = correct/len(wine_data.data)
print(f'correct predictions: {correct}')
print(f'accuracy using 6-fold-cross-validation: {acc}')
        

correct predictions: 105.0
accuracy using 6-fold-cross-validation: 0.5898876404494382
